# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 26 2022 1:37PM,242770,19,ACG-2102490723,ACG North America LLC,Released
1,Jul 26 2022 1:19PM,242768,10,CTF0011337,"Citieffe, Inc.",Released
2,Jul 26 2022 1:16PM,242767,10,CLI00343.1,"CLINUVEL, Inc.",Released
3,Jul 26 2022 1:13PM,242765,20,8326031,"ACI Healthcare USA, Inc.",Released
4,Jul 26 2022 1:13PM,242765,20,8326042,"ACI Healthcare USA, Inc.",Executing
5,Jul 26 2022 1:13PM,242765,20,8326046,"ACI Healthcare USA, Inc.",Released
6,Jul 26 2022 1:01PM,242763,19,8326049,"Avet Pharmaceuticals Labs, Inc",Executing
7,Jul 26 2022 12:53PM,242761,22,622121,"NBTY Global, Inc.",Released
8,Jul 26 2022 12:53PM,242760,22,622120,"NBTY Global, Inc.",Released
9,Jul 26 2022 12:53PM,242759,10,8326039,"ClearSpec, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,242765,Executing,1
31,242765,Released,2
32,242767,Released,1
33,242768,Released,1
34,242770,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242763,NaN,1.0,NaN
242765,NaN,1.0,2.0
242767,NaN,NaN,1.0
242768,NaN,NaN,1.0
242770,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242696,45.0,8.0,0.0
242711,15.0,10.0,0.0
242718,10.0,23.0,6.0
242720,0.0,0.0,50.0
242721,9.0,5.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242696,45,8,0
242711,15,10,0
242718,10,23,6
242720,0,0,50
242721,9,5,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242696,45,8,0
1,242711,15,10,0
2,242718,10,23,6
3,242720,0,0,50
4,242721,9,5,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242696,45,8,
1,242711,15,10,
2,242718,10,23,6
3,242720,,,50
4,242721,9,5,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 26 2022 1:37PM,242770,19,ACG North America LLC
1,Jul 26 2022 1:19PM,242768,10,"Citieffe, Inc."
2,Jul 26 2022 1:16PM,242767,10,"CLINUVEL, Inc."
3,Jul 26 2022 1:13PM,242765,20,"ACI Healthcare USA, Inc."
6,Jul 26 2022 1:01PM,242763,19,"Avet Pharmaceuticals Labs, Inc"
7,Jul 26 2022 12:53PM,242761,22,"NBTY Global, Inc."
8,Jul 26 2022 12:53PM,242760,22,"NBTY Global, Inc."
9,Jul 26 2022 12:53PM,242759,10,"ClearSpec, LLC"
11,Jul 26 2022 12:15PM,242755,10,Emerginnova
15,Jul 26 2022 12:08PM,242753,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 26 2022 1:37PM,242770,19,ACG North America LLC,,,1
1,Jul 26 2022 1:19PM,242768,10,"Citieffe, Inc.",,,1
2,Jul 26 2022 1:16PM,242767,10,"CLINUVEL, Inc.",,,1
3,Jul 26 2022 1:13PM,242765,20,"ACI Healthcare USA, Inc.",,1,2
4,Jul 26 2022 1:01PM,242763,19,"Avet Pharmaceuticals Labs, Inc",,1,
5,Jul 26 2022 12:53PM,242761,22,"NBTY Global, Inc.",,,1
6,Jul 26 2022 12:53PM,242760,22,"NBTY Global, Inc.",,,1
7,Jul 26 2022 12:53PM,242759,10,"ClearSpec, LLC",,,2
8,Jul 26 2022 12:15PM,242755,10,Emerginnova,,,4
9,Jul 26 2022 12:08PM,242753,10,"Methapharm, Inc.",,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 1:37PM,242770,19,ACG North America LLC,1,,
1,Jul 26 2022 1:19PM,242768,10,"Citieffe, Inc.",1,,
2,Jul 26 2022 1:16PM,242767,10,"CLINUVEL, Inc.",1,,
3,Jul 26 2022 1:13PM,242765,20,"ACI Healthcare USA, Inc.",2,1,
4,Jul 26 2022 1:01PM,242763,19,"Avet Pharmaceuticals Labs, Inc",,1,
5,Jul 26 2022 12:53PM,242761,22,"NBTY Global, Inc.",1,,
6,Jul 26 2022 12:53PM,242760,22,"NBTY Global, Inc.",1,,
7,Jul 26 2022 12:53PM,242759,10,"ClearSpec, LLC",2,,
8,Jul 26 2022 12:15PM,242755,10,Emerginnova,4,,
9,Jul 26 2022 12:08PM,242753,10,"Methapharm, Inc.",3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 1:37PM,242770,19,ACG North America LLC,1,,
1,Jul 26 2022 1:19PM,242768,10,"Citieffe, Inc.",1,,
2,Jul 26 2022 1:16PM,242767,10,"CLINUVEL, Inc.",1,,
3,Jul 26 2022 1:13PM,242765,20,"ACI Healthcare USA, Inc.",2,1,
4,Jul 26 2022 1:01PM,242763,19,"Avet Pharmaceuticals Labs, Inc",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 1:37PM,242770,19,ACG North America LLC,1.0,NaN,NaN
1,Jul 26 2022 1:19PM,242768,10,"Citieffe, Inc.",1.0,NaN,NaN
2,Jul 26 2022 1:16PM,242767,10,"CLINUVEL, Inc.",1.0,NaN,NaN
3,Jul 26 2022 1:13PM,242765,20,"ACI Healthcare USA, Inc.",2.0,1.0,NaN
4,Jul 26 2022 1:01PM,242763,19,"Avet Pharmaceuticals Labs, Inc",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 26 2022 1:37PM,242770,19,ACG North America LLC,1.0,0.0,0.0
1,Jul 26 2022 1:19PM,242768,10,"Citieffe, Inc.",1.0,0.0,0.0
2,Jul 26 2022 1:16PM,242767,10,"CLINUVEL, Inc.",1.0,0.0,0.0
3,Jul 26 2022 1:13PM,242765,20,"ACI Healthcare USA, Inc.",2.0,1.0,0.0
4,Jul 26 2022 1:01PM,242763,19,"Avet Pharmaceuticals Labs, Inc",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2184710,29.0,18.0,48.0
15,485463,10.0,0.0,1.0
16,485442,0.0,10.0,18.0
19,728280,9.0,8.0,4.0
20,1213637,89.0,34.0,25.0
21,242746,1.0,0.0,0.0
22,485521,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2184710,29.0,18.0,48.0
1,15,485463,10.0,0.0,1.0
2,16,485442,0.0,10.0,18.0
3,19,728280,9.0,8.0,4.0
4,20,1213637,89.0,34.0,25.0
5,21,242746,1.0,0.0,0.0
6,22,485521,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,29.0,18.0,48.0
1,15,10.0,0.0,1.0
2,16,0.0,10.0,18.0
3,19,9.0,8.0,4.0
4,20,89.0,34.0,25.0
5,21,1.0,0.0,0.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,29.0
1,15,Released,10.0
2,16,Released,0.0
3,19,Released,9.0
4,20,Released,89.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Completed,48.0,1.0,18.0,4.0,25.0,0.0,0.0
Executing,18.0,0.0,10.0,8.0,34.0,0.0,0.0
Released,29.0,10.0,0.0,9.0,89.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Completed,48.0,1.0,18.0,4.0,25.0,0.0,0.0
1,Executing,18.0,0.0,10.0,8.0,34.0,0.0,0.0
2,Released,29.0,10.0,0.0,9.0,89.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Completed,48.0,1.0,18.0,4.0,25.0,0.0,0.0
1,Executing,18.0,0.0,10.0,8.0,34.0,0.0,0.0
2,Released,29.0,10.0,0.0,9.0,89.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()